# Playing with CKKS
> Based on https://blog.openmined.org/ckks-explained-part-3-encryption-and-decryption/ 
- toc: true
- branch: master
- badges: true
- comments: true
- author: Conwyn
- categories: [fastai,FHE,CKKS]

CKSS is a method of taking Complex (so Real) numbers and generating polynomials and then encrypting those polynomials. It is possible to add and multiply these encrypted polynomials and then decrypt them giving the answers as if the same operation applied to the original polynomials and the actual initial Real numbers. So ideal for deep learning matrix multipliction.

In [ ]:
from numpy.polynomial import Polynomial
import numpy as np

def round_coordinates(coordinates):
    """Gives the integral rest."""
    coordinates = coordinates - np.floor(coordinates)
    return coordinates

def coordinate_wise_random_rounding(coordinates):
    """Rounds coordinates randonmly."""
    r = round_coordinates(coordinates)
    f = np.array([np.random.choice([c, c-1], 1, p=[1-c, c]) for c in r]).reshape(-1)
    
    rounded_coordinates = coordinates - f
    rounded_coordinates = [int(coeff) for coeff in rounded_coordinates]
    return rounded_coordinates

class CKKSEncoder:
    """Basic CKKS encoder to encode complex vectors into polynomials."""
    
    def __init__(self, M:int, scale:float):
        """Initializes with scale."""
        self.xi = np.exp(2 * np.pi * 1j / M)
        self.M = M
        self.create_sigma_R_basis()
        self.scale = scale
        
    @staticmethod
    def vandermonde(xi: np.complex128, M: int) -> np.array:
        """Computes the Vandermonde matrix from a m-th root of unity."""
        
        N = M //2
        matrix = []
        # We will generate each row of the matrix
        for i in range(N):
            # For each row we select a different root
            root = xi ** (2 * i + 1)
            row = []

            # Then we store its powers
            for j in range(N):
                row.append(root ** j)
            matrix.append(row)
        return matrix
    
    def sigma_inverse(self, b: np.array) -> Polynomial:
        """Encodes the vector b in a polynomial using an M-th root of unity."""

        # First we create the Vandermonde matrix
        A = CKKSEncoder.vandermonde(self.xi, M)

        # Then we solve the system
        coeffs = np.linalg.solve(A, b)

        # Finally we output the polynomial
        p = Polynomial(coeffs)
        return p

    def sigma(self, p: Polynomial) -> np.array:
        """Decodes a polynomial by applying it to the M-th roots of unity."""

        outputs = []
        N = self.M //2

        # We simply apply the polynomial on the roots
        for i in range(N):
            root = self.xi ** (2 * i + 1)
            output = p(root)
            outputs.append(output)
        return np.array(outputs)
    

    def pi(self, z: np.array) -> np.array:
        """Projects a vector of H into C^{N/2}."""

        N = self.M // 4
        return z[:N]


    def pi_inverse(self, z: np.array) -> np.array:
        """Expands a vector of C^{N/2} by expanding it with its
        complex conjugate."""

        z_conjugate = z[::-1]
        z_conjugate = [np.conjugate(x) for x in z_conjugate]
        return np.concatenate([z, z_conjugate])
    
    def create_sigma_R_basis(self):
        """Creates the basis (sigma(1), sigma(X), ..., sigma(X** N-1))."""

        self.sigma_R_basis = np.array(self.vandermonde(self.xi, self.M)).T
    

    def compute_basis_coordinates(self, z):
        """Computes the coordinates of a vector with respect to the orthogonal lattice basis."""
        output = np.array([np.real(np.vdot(z, b) / np.vdot(b,b)) for b in self.sigma_R_basis])
        return output

    def sigma_R_discretization(self, z):
        """Projects a vector on the lattice using coordinate wise random rounding."""
        coordinates = self.compute_basis_coordinates(z)

        rounded_coordinates = coordinate_wise_random_rounding(coordinates)
        y = np.matmul(self.sigma_R_basis.T, rounded_coordinates)
        return y


    def encode(self, z: np.array) -> Polynomial:
        """Encodes a vector by expanding it first to H,
        scale it, project it on the lattice of sigma(R), and performs
        sigma inverse.
        """
        pi_z = self.pi_inverse(z)
        scaled_pi_z = self.scale * pi_z
        rounded_scale_pi_zi = self.sigma_R_discretization(scaled_pi_z)
        p = self.sigma_inverse(rounded_scale_pi_zi)

        # We round it afterwards due to numerical imprecision
        coef = np.round(np.real(p.coef)).astype(int)
        p = Polynomial(coef)
        return p


    def decode(self, p: Polynomial) -> np.array:
        """Decodes a polynomial by removing the scale, 
        evaluating on the roots, and project it on C^(N/2)"""
        rescaled_p = p / self.scale
        z = self.sigma(rescaled_p)
        pi_z = self.pi(z)
        return pi_z



M = 8
N = M //2
scale = 64

# We set xi, which will be used in our computations
xi = np.exp(2 * np.pi * 1j / M)
xi
encoder = CKKSEncoder(M,scale)

In [ ]:
# First we set the parameters
M = 8
N = M //2

# We set xi, which will be used in our computations
xi = np.exp(2 * np.pi * 1j / M)
xi


(0.7071067811865476+0.7071067811865475j)

In [ ]:

b = np.array([1, 2, 3, 4])
b

p = encoder.sigma_inverse(b)
p

b_reconstructed = encoder.sigma(p)
b_reconstructed



np.linalg.norm(b_reconstructed - b)

6.944442800358888e-16

In [ ]:
z = np.array([3 +4j, 2 - 1j])
z

p = encoder.encode(z)
print(p)



encoder.decode(p)

poly([160.  90. 160.  45.])


array([2.99718446+3.99155337j, 2.00281554-1.00844663j])

In [ ]:
z = np.array([1,0])
z

p = encoder.encode(z)
print(p)



encoder.decode(p)

poly([ 32.  22.   0. -23.])


array([0.99718446-0.01104854j, 0.00281554-0.01104854j])

In [ ]:
z = np.array([2,0])
z

p = encoder.encode(z)
print(p)



encoder.decode(p)

poly([ 64.  46.   0. -46.])


array([ 2.016466+0.0000000e+00j, -0.016466+4.4408921e-16j])

In [ ]:
z = np.array([3,0])
z

p = encoder.encode(z)
print(p)



encoder.decode(p)

poly([ 96.  68.   0. -67.])


array([2.99155337+0.01104854j, 0.00844663+0.01104854j])

In [ ]:
z = np.array([4,0])
z

p = encoder.encode(z)
print(p)



encoder.decode(p)

poly([128.  90.   0. -91.])


array([3.99978637e+00-0.01104854j, 2.13634457e-04-0.01104854j])

In [ ]:
z = np.array([5,0])
z

p = encoder.encode(z)
print(p)



encoder.decode(p)

poly([ 160.  113.    0. -113.])


array([4.99697082e+00-4.44089210e-16j, 3.02917894e-03+1.33226763e-15j])

In [ ]:
z = np.array([1,0])
z

p = encoder.encode(z)
print(p)



encoder.decode(p)

poly([ 32.  23.   0. -23.])


array([ 1.008233+0.00000000e+00j, -0.008233+2.22044605e-16j])

In [ ]:
a1=Polynomial([32,22,0,-23])
print(a1)
a2 = 2 * a1
print(a2)
encoder.decode(a2)

poly([ 32.  22.   0. -23.])
poly([ 64.  44.   0. -46.])


array([1.99436891-0.02209709j, 0.00563109-0.02209709j])

In [ ]:
import numpy as np
from numpy.polynomial import polynomial as poly

def polymul(x, y, modulus, poly_mod):
   
     """Multiply two polynoms
    Args:
        x, y: two polynoms to be multiplied.
        modulus: coefficient modulus.
        poly_mod: polynomial modulus.
    Returns:
        A polynomial in Z_modulus[X]/(poly_mod).
    """
    return np.int64(
        np.round(poly.polydiv(poly.polymul(x, y) % modulus, poly_mod)[1] % modulus)
    )


def polyadd(x, y, modulus, poly_mod):
    """Add two polynoms
    Args:
        x, y: two polynoms to be added.
        modulus: coefficient modulus.
        poly_mod: polynomial modulus.
    Returns:
        A polynomial in Z_modulus[X]/(poly_mod).
    """
    return np.int64(
        np.round(poly.polydiv(poly.polyadd(x, y) % modulus, poly_mod)[1] % modulus)
    )


In [ ]:
def gen_binary_poly(size):
    """Generates a polynomial with coeffecients in [0, 1]
    Args:
        size: number of coeffcients, size-1 being the degree of the
            polynomial.
    Returns:
        array of coefficients with the coeff[i] being 
        the coeff of x ^ i.
    """
    return np.random.randint(0, 2, size, dtype=np.int64)


def gen_uniform_poly(size, modulus):
    """Generates a polynomial with coeffecients being integers in Z_modulus
    Args:
        size: number of coeffcients, size-1 being the degree of the
            polynomial.
    Returns:
        array of coefficients with the coeff[i] being 
        the coeff of x ^ i.
    """
    return np.random.randint(0, modulus, size, dtype=np.int64)


def gen_normal_poly(size):
    """Generates a polynomial with coeffecients in a normal distribution
    of mean 0 and a standard deviation of 2, then discretize it.
    Args:
        size: number of coeffcients, size-1 being the degree of the
            polynomial.
    Returns:
        array of coefficients with the coeff[i] being 
        the coeff of x ^ i.
    """
    return np.int64(np.random.normal(0, 2, size=size))


In [ ]:
def keygen(size, modulus, poly_mod):
    """Generate a public and secret keys
    Args:
        size: size of the polynoms for the public and secret keys.
        modulus: coefficient modulus.
        poly_mod: polynomial modulus.
    Returns:
        Public and secret key.
    
    sk = gen_binary_poly(size)
    a = gen_uniform_poly(size, modulus)
    e = gen_normal_poly(size)
    """
    b = polyadd(polymul(-a, sk, modulus, poly_mod), -e, modulus, poly_mod)
    return (b, a), sk

In [ ]:
def encrypt(pk, size, q, t, poly_mod, pt):
    """Encrypt an integer.
    Args:
        pk: public-key.
        size: size of polynomials.
        q: ciphertext modulus.
        t: plaintext modulus.
        poly_mod: polynomial modulus.
        pt: integer to be encrypted.
    Returns:
        Tuple representing a ciphertext.      
    """
    """
    e1 = gen_normal_poly(size)
    e2 = gen_normal_poly(size)
    u = gen_binary_poly(size)
    """
    # encode the integer into a plaintext polynomial
    # m = np.array([pt] + [0] * (size - 1), dtype=np.int64) % t
    m = pt %t
    delta = q // t
    scaled_m = m * delta  % q
    
    ct0 = polyadd(
            polyadd(
                polymul(pk[0], u, q, poly_mod),
                e1, q, poly_mod),
            scaled_m, q, poly_mod
        )
    ct1 = polyadd(
            polymul(pk[1], u, q, poly_mod),
            e2, q, poly_mod
        )
    return (ct0, ct1)


In [ ]:
def decrypt(sk, size, q, t, poly_mod, ct):
    """Decrypt a ciphertext
    Args:
        sk: secret-key.
        size: size of polynomials.
        q: ciphertext modulus.
        t: plaintext modulus.
        poly_mod: polynomial modulus.
        ct: ciphertext.
    Returns:
        Integer representing the plaintext.#   
        """
    scaled_pt = polyadd(
            polymul(ct[1], sk, q, poly_mod),
            ct[0], q, poly_mod
        )
    decrypted_poly = np.round(scaled_pt * t / q) % t
    # return int(decrypted_poly[0])
    return  (decrypted_poly)


In [ ]:
size=2**2
e1 = gen_normal_poly(size)
e2 = gen_normal_poly(size)
u = gen_binary_poly(size)
# polynomial modulus degree
n = 2**2
# ciphertext modulus
q = 2**15
# plaintext modulus
t = 2**8
# polynomial modulus
poly_mod = np.array([1] + [0] * (n - 1) + [1])
# Keygen
sk = gen_binary_poly(size)
modulus = 2**15
a = gen_uniform_poly(size, modulus)
e = gen_normal_poly(size)
pk, sk = keygen(n, q, poly_mod)

In [ ]:
ct1 = encrypt(pk, n, q, t, poly_mod, np.array([ 32,22,   0, -23]))
print(ct1)
decrypted_ct1 = decrypt(sk, n, q, t, poly_mod, ct1)
print(decrypted_ct1)

(array([29281, 29854,  2994, 15207]), array([23343,  7572, 16473,  1144]))
[ 32.  22.   0. 233.]


In [ ]:
ct2= (ct1[0] + ct1[0],ct1[0]+ct1[0])
print(ct2)
decrypted_ct2 = decrypt(sk, n, q, t, poly_mod, ct2)
print(decrypted_ct2)

(array([58562, 59708,  5988, 30414]), array([58562, 59708,  5988, 30414]))
[219. 128. 221. 184.]


In [ ]:
def gen_binary_poly(size):
    """Generates a polynomial with coeffecients in [0, 1]
    Args:
        size: number of coeffcients, size-1 being the degree of the
            polynomial.
    Returns:
        array of coefficients with the coeff[i] being 
        the coeff of x ^ i.
    """
    return np.random.randint(0, 2, size, dtype=np.int64)


def gen_uniform_poly(size, modulus):
    """Generates a polynomial with coeffecients being integers in Z_modulus
    Args:
        size: number of coeffcients, size-1 being the degree of the
            polynomial.
    Returns:
        array of coefficients with the coeff[i] being 
        the coeff of x ^ i.
    """
    return np.random.randint(0, modulus, size, dtype=np.int64)


def gen_normal_poly(size):
    """Generates a polynomial with coeffecients in a normal distribution
    of mean 0 and a standard deviation of 2, then discretize it.
    Args:
        size: number of coeffcients, size-1 being the degree of the
            polynomial.
    Returns:
        array of coefficients with the coeff[i] being 
        the coeff of x ^ i.
    """



In [ ]:
    np.int64(np.random.normal(0, 2, size=5))

array([ 0,  0,  2, -1, -2])

In [ ]:

import math
np.random.normal(0,math.sqrt(2),5)

array([-0.15210849, -1.40642172, -0.50763401, -0.31183585,  1.27311431])

In [ ]:
an_array = np.array([1, 2, 3])

def double(x):

  return x * 2


mapped_array = double(an_array)


print(mapped_array)

[2 4 6]


In [ ]:

from numpy.polynomial import polynomial as poly
import numpy as np
import math
# Create a function that adds 100 to something
add_100 = lambda i: i + 100

# Create a vectorized function


def randomZO(x) :
  lookup = [-1,0,0,1]
  return lookup[x]
  #given p=0.5 create q = 1/p so 2 create array 2q with middle elements 0 and 0 otherwise -1 and 1. change rndint to be 0,2q
  #So p = 0.25 q = 4 array 11100111

def randomZOinverse(x) :
  if x == 0 :
         return 0
  return 1
  

qmod = 2**10
P = 2**8

poly_mod = np.array([1,0,0,0,1])
def ZOgenerator(  ):
  vectorized_ZO = np.vectorize(randomZO)
  vectorized_ZOinverse = np.vectorize(randomZOinverse)
  sa = vectorized_ZO(np.random.randint(0,4,4))
  sai = vectorized_ZOinverse(sa)
  while not((2 == (sai == 1).sum()) ) :
    sa = vectorized_ZO(np.random.randint(0,4,4))
    sai = vectorized_ZOinverse(sa)
  return sa
  
s = ZOgenerator()
sk = (1,s) #

ea = np.random.normal(0,math.sqrt(2),4)
e = (ea)
qmodsmall = 5
aa = np.random.randint(0,qmodsmall,4)
a = (aa)


b1 = poly.polymul(a,s)

b2q,b2r = poly.polydiv(b1%qmod,poly_mod) 
#print("b2r")
#print(b2r)
b3 = poly.polyadd(e,-b2r%qmod)
b4q,b4r = poly.polydiv(b3%qmod,poly_mod)
b = b4r
pk = (b,a)

#validation key
PQ = P*qmod

av = np.random.randint(0,qmodsmall,4)
ev = np.random.normal(0,math.sqrt(2),4)
b1 = poly.polymul(av,s)

b2q,b2r = poly.polydiv(b1%PQ,poly_mod) 
#print("b2r")
#print(b2r)
b3 = poly.polyadd(e,-b2r%PQ)
b4q,b4r = poly.polydiv(b3%PQ,poly_mod)
Ps2 = P*(poly.polymul(s,s))
b5 = poly.polyadd(b3,Ps2)  #br4
b6q,b6r = poly.polydiv(b5%PQ,poly_mod)
bv = b6r
evk = (bv,av)


#encrypt

def ckksencrypt(m):
  v = ZOgenerator()
  e0 = np.random.normal(0,math.sqrt(2),4)
  e1 = np.random.normal(0,math.sqrt(2),4)
  c0a = poly.polymul(v,b)
  c0b = poly.polyadd(c0a,e0)
  c0c = poly.polyadd(c0b,m)
  c0q,c0r = poly.polydiv(c0c%qmod,poly_mod)
  c0 = c0r%qmod
  c1a = poly.polymul(v,a)
  c1b = poly.polyadd(c1a,e1)
  c1q,c1r = poly.polydiv(c1b%qmod,poly_mod)
  c1 = c1r%qmod 
  return (c0,c1)

#decrypt
def ckksdecrypt(c0,c1,s) :
 dq,dr=poly.polydiv(poly.polyadd(c0,poly.polymul(c1,s))%qmod,poly_mod)
 dm = dr % qmod
 #print("s a b c0 c1 dm")
 #print(s,a,b)
 #print(c0,c1)
 #print(dm)
 return (np.around(dm))
#
m0 = np.array([30,40,50,60])
m2 = np.array([60,50,40,30])

gc0,gc1 = ckksencrypt(m0)
gc2,gc3 = ckksencrypt(m2)
gs = s
gc4 = np.add(gc0,gc2)
gc5 = np.add(gc1,gc3)
print(ckksdecrypt(gc0,gc1,gs))
print(ckksdecrypt(gc2,gc3,gs))
print(ckksdecrypt(gc4,gc5,gs))
d0m = poly.polymul(gc0,gc2)
d0q,d0r = poly.polydiv(d0m,poly_mod)
d0 = d0r
d1ma = poly.polymul(gc0,gc3)
d1mb = poly.polymul(gc1,gc2)
d1m =poly.polyadd(d1ma,d1mb)
d1q,d1r = poly.polydiv(d1m,poly_mod)
d1 = d1r
d2m = poly.polymul(gc1,gc3)
d2q,d2r = poly.polydiv(d2m,poly_mod)
d2 = d2r
print(d0,d1,d2)
print ("evk")
print (evk)
d2evk0 = poly.polymul(d2,bv)%qmod
d2evk1 = poly.polymul(d2,av)%qmod
d2evk0q,d2evk0r = (poly.polydiv(d2evk0,poly_mod))
d2evk1q,d2evk1r = (poly.polydiv(d2evk1,poly_mod))
print("d2ev")
print(d2evk0r,d2evk1r)
d2a0 = poly.polyadd(d0,d2evk0r/P)
d2a1 = poly.polyadd(d1,d2evk1r/P)
d2a0q,d2a0r = poly.polydiv(d2a0%qmod,poly_mod)
d2a1q,d2a1r = poly.polydiv(d2a1%qmod,poly_mod)
gc6 =d2a0r
gc7 = d2a1r
print("decrypt mult")
print(ckksdecrypt(gc6,gc7,gs))

[27. 38. 47. 62.]
[59. 52. 35. 33.]
[85. 90. 83. 95.]
[-4256.04489788   119.78987288  4447.73982358  8308.64367718] [-64520.64643198  24433.53648975 125827.70150041 143102.38467171] [-7066.59732099  4688.89450294 15407.15132969 15943.79869264]
evk
(array([ 2.61631816e+05,  2.62142369e+05, -1.98992018e+00,  2.62143947e+05]), array([1, 1, 0, 0]))
d2ev
[-225.80957222 -931.77543116   16.27155685  244.74714661] [-482.39601362  694.29718196  640.04583264  630.95002233]
decrypt mult
[ 234.  227.  467. 1009.]
